In [1]:
import logging

from logger_config import setup_logging

setup_logging(level=logging.INFO)


import configparser

import pandas as pd
import main

from economic_data.extract.economic_data import (
    fetch_ecb_json,
    fetch_eurostat_json,
    fetch_fred_json,
    get_historical_stock_data,
)
from economic_data.transform.transform_economic_data import (
    calculate_monthly_change,
    ecb_json_to_df,
    eurostat_json_to_df,
    fred_json_to_df,
    label_and_append,
    set_monthly_ecb_interest_rate,
    rename_economic_indicators,
    threshold_csv_to_df,
    load_thresholds,
)
from economic_data.transform.transform_stockmarket_data import (
    convert_google_finance_index_to_dict,
    convert_google_finance_data_to_dict,
)

from economic_data.transform.transform_economic_data import (
    convert_eurostat_infl_ind_to_dict,
    # convert_eurostat_unemployment_to_dict,
    # convert_eurostat_gdp_to_dict,
)

from economic_data.load.save_data import (
    save_stock_index,
    save_stock_data,
    save_economic_indicator,
)

logger = logging.getLogger(__name__)
pd.set_option("display.max_rows", 500)

In [2]:
dict = main.fetch_eurostat_json("prc_hicp_mmor", "2020-01-01")
dict

2025-06-11 22:47:20,918 - INFO - fetch_eurostat_json - Fetching Eurostat data for prc_hicp_mmor and from 2020-01


{'version': '2.0',
 'class': 'dataset',
 'label': 'HICP - monthly data (monthly rate of change)',
 'source': 'ESTAT',
 'updated': '2025-06-03T11:00:00+0200',
 'value': {'29835': 0.4,
  '29836': 0.3,
  '29837': 0.2,
  '29838': -0.5,
  '29839': 0.0,
  '29840': 0.1,
  '29841': -0.2,
  '29842': 0.1,
  '29843': 0.2,
  '29844': 0.3,
  '29845': 0.2,
  '29846': 0.1,
  '29847': 1.0,
  '29848': 0.3,
  '29849': 0.1,
  '29850': 0.2,
  '29851': 0.1,
  '29852': 0.2,
  '29853': 0.6,
  '29854': 0.1,
  '29855': 0.3,
  '29856': 1.1,
  '29857': 0.6,
  '29858': 0.4,
  '29859': 2.1,
  '29860': 0.3,
  '29861': 1.5,
  '29862': 0.2,
  '29863': -0.2,
  '29864': -1.2,
  '29865': 0.9,
  '29866': 1.4,
  '29867': 2.8,
  '29868': 1.2,
  '29869': -0.6,
  '29870': -0.1,
  '29871': 2.4,
  '29872': 0.2,
  '29873': 0.0,
  '29874': 0.4,
  '29875': -0.4,
  '29876': -0.2,
  '29877': -0.2,
  '29878': 0.4,
  '29879': 0.3,
  '29880': 0.2,
  '29881': 0.0,
  '29882': 0.1,
  '29883': 1.6,
  '29884': 0.7,
  '29885': 0.0,
  '29886

In [3]:
tmp = convert_eurostat_infl_ind_to_dict(dict, "namn_test", "beskrivning")
tmp

2025-06-11 22:47:22,392 - INFO - convert_eurostat_infl_ind_to_dict - Creating index data for namn_test


{'indicator_id': 'HICP - monthly data (monthly rate of change)',
 'name': 'namn_test',
 'description': 'beskrivning',
 'unit': 'beskrivning',
 'frequency': <Frequency.monthly: 'monthly'>,
 'source': 'ESTAT'}

In [6]:
try:
    save_economic_indicator(tmp)
except Exception as e:
    print(f"Could not save economic indicator: {e}")

In [2]:
omx_smi_dict = get_historical_stock_data(
    "INDEXNASDAQ:OMXSPI", main.SERVICE_ACCOUNT_FILE, main.SPREADSHEET_ID, main.FROM_DATE
)
index_omx = convert_google_finance_index_to_dict(
    omx_smi_dict, "omx_smi", "stokcholms index", "google spreadsheet"
)

index_omx_id = save_stock_index(
    index_omx,
)

data_omx = convert_google_finance_data_to_dict(omx_smi_dict)

save_stock_data(index_omx_id, data_omx)

Inserted 2065 new records.


In [3]:
omx_smi_dict[:3]

[['Ticker_id', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'],
 ['INDEXNASDAQ:OMXSPI',
  '2017-01-02 16.00.00',
  '534,23',
  '539,37',
  '533,16',
  '538,38',
  '0'],
 ['INDEXNASDAQ:OMXSPI',
  '2017-01-03 16.00.00',
  '539,35',
  '541,7',
  '538,31',
  '539,74',
  '0']]

In [3]:
save_stock_data(index_omx_id, data_omx)

Inserted 0 new records.


In [5]:
"""

id = Column(Integer, primary_key=True)
ticker_id = Column(
    String, unique=True, nullable=False
)  # unique ticker/symbol like 'INDEXNASDAQ:OMXSPI'
name = Column(
    String, nullable=False
)  # descriptive name like 'OMX Stockholm All-Share Index'
description = Column(String)
source = Column(String)

"""

omx_smi_dict = get_historical_stock_data(
    "INDEXNASDAQ:OMXSPI", main.SERVICE_ACCOUNT_FILE, main.SPREADSHEET_ID, main.FROM_DATE
)

index_dict = {
    "ticker_id": "INDEXNASDAQ:OMXSPI",
    "name": "OMX Stockholm All-Share Index",
    "description": "The OMX Stockholm All-Share Index (OMXSPI) is a stock market index that tracks the performance of all common stocks listed on the Nasdaq Stockholm exchange.",
    "source": "Google Finance",
}
# NEXT: Transform-filen skapar rätt format på en dict för save_data-metoden

save_stock_index(
    index_dict,
)

Stock index with ticker_id 'INDEXNASDAQ:OMXSPI' already exists. Skipping.


1

In [6]:
symbol = "STO:LATO-B"  # INDEXNASDAQ:OMXSPI, INDEXSP:.INX
SERVICE_ACCOUNT_FILE = "config/stock-data-462106-4b3b621b4a82.json"  # Rename and place your downloaded JSON key here

# The ID of your spreadsheet
SPREADSHEET_ID = "1Ih_rIE-woq0gkrBd5snyl8IxJP0GtM2iVKdPP53TRsM"  # https://docs.google.com/spreadsheets/d/1Ih_rIE-woq0gkrBd5snyl8IxJP0GtM2iVKdPP53TRsM/edit?gid=0#gid=0


file_path = "data/stock_data.csv"
data = get_historical_stock_data(
    symbol,
    SERVICE_ACCOUNT_FILE,
    SPREADSHEET_ID,
)
data

[['Ticker_id', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'],
 ['STO:LATO-B', '2020-01-02 17.30.00', '154', '157,3', '154', '157', '360738'],
 ['STO:LATO-B',
  '2020-01-03 17.30.00',
  '157',
  '157,3',
  '154,6',
  '156,4',
  '427673'],
 ['STO:LATO-B',
  '2020-01-07 17.30.00',
  '156',
  '157,2',
  '154,1',
  '155,7',
  '449726'],
 ['STO:LATO-B',
  '2020-01-08 17.30.00',
  '154,9',
  '156,7',
  '153,3',
  '156,7',
  '301045'],
 ['STO:LATO-B',
  '2020-01-09 17.30.00',
  '157,7',
  '158,8',
  '156,9',
  '157,6',
  '338930'],
 ['STO:LATO-B',
  '2020-01-10 17.30.00',
  '158',
  '158,3',
  '156,4',
  '156,4',
  '261346'],
 ['STO:LATO-B',
  '2020-01-13 17.30.00',
  '157',
  '157,8',
  '156,1',
  '157',
  '186665'],
 ['STO:LATO-B',
  '2020-01-14 17.30.00',
  '157,3',
  '158',
  '155,8',
  '157',
  '355787'],
 ['STO:LATO-B',
  '2020-01-15 17.30.00',
  '157',
  '157,5',
  '155,5',
  '156,3',
  '491879'],
 ['STO:LATO-B',
  '2020-01-16 17.30.00',
  '156,3',
  '158,5',
  '156,3',
  '158,4',
 

In [7]:
data

[['Ticker_id', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'],
 ['STO:LATO-B', '2020-01-02 17.30.00', '154', '157,3', '154', '157', '360738'],
 ['STO:LATO-B',
  '2020-01-03 17.30.00',
  '157',
  '157,3',
  '154,6',
  '156,4',
  '427673'],
 ['STO:LATO-B',
  '2020-01-07 17.30.00',
  '156',
  '157,2',
  '154,1',
  '155,7',
  '449726'],
 ['STO:LATO-B',
  '2020-01-08 17.30.00',
  '154,9',
  '156,7',
  '153,3',
  '156,7',
  '301045'],
 ['STO:LATO-B',
  '2020-01-09 17.30.00',
  '157,7',
  '158,8',
  '156,9',
  '157,6',
  '338930'],
 ['STO:LATO-B',
  '2020-01-10 17.30.00',
  '158',
  '158,3',
  '156,4',
  '156,4',
  '261346'],
 ['STO:LATO-B',
  '2020-01-13 17.30.00',
  '157',
  '157,8',
  '156,1',
  '157',
  '186665'],
 ['STO:LATO-B',
  '2020-01-14 17.30.00',
  '157,3',
  '158',
  '155,8',
  '157',
  '355787'],
 ['STO:LATO-B',
  '2020-01-15 17.30.00',
  '157',
  '157,5',
  '155,5',
  '156,3',
  '491879'],
 ['STO:LATO-B',
  '2020-01-16 17.30.00',
  '156,3',
  '158,5',
  '156,3',
  '158,4',
 